In [ ]:
#!pip install rank_bm25

In [65]:
import random
import re
import numpy as np
from rank_bm25 import *

In [66]:
#adding words to stopwords
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [67]:
%%markdown
# Load and process CISI dataset 

# Load and process CISI dataset 


In [68]:
dataset_folder_path = '/content/drive/MyDrive/doutorado/P_IA368DD_2023S/t1/cisi/'

# Processing all documents 
doc_set = {}
doc_id = ""
doc_text = ""
with open(dataset_folder_path+'CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
doc_count = 0
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.    

        
# Processing queries
with open(dataset_folder_path+'CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

# Processing Relevance assesments
rel_set = {}
with open(dataset_folder_path+'CISI.REL') as f:
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

In [69]:
random.seed(13)
idx = random.sample(rel_set.keys(),1)[0]

print('Query ID %s ==>' % idx, qry_set[idx])
rel_docs = rel_set[idx]
print('Documents relevants to Query ID %s' % idx, rel_docs)
sample_document_idx = random.sample(rel_docs,1)[0]
print('Document ID %s ==>' % sample_document_idx, doc_set[sample_document_idx])

Query ID 33 ==> Methods of coding used in computerized index systems.
Documents relevants to Query ID 33 [33, 43, 52, 56, 67, 79, 116, 122, 158, 319, 376, 433, 488, 492, 521, 529, 599, 667, 668, 669, 670, 672, 676, 677, 678, 680, 686, 688, 693, 696, 698, 699, 703, 705, 707, 708, 829, 1143]
Document ID 668 ==> Rapid Structure Searches via Permuted Chemical Line-Notations Sorter, P.F. Granito, C.E. Gilmer, J.C. Gelberg, A. Metcalf, E.A. The Wiswesser chemical line-notation is an unique and unambiguous method of representing chemical structures by a linear series of letters, numbers, ampersands, and hyphens.  These symbols are meaningful to chemists familiar with the notation and can be processed by automatic data processing (ADP) equipment. The uniqueness of the line-notation permits the use of alphanumerically arranged lists of notations for dictionary-type searches.  This ordered arrangement permits the rapid location of a specific compound or a specific class of ring compounds other t

In [70]:
#Removing special characters:
def special_chars_removal(lst):
    lst1=list()
    for element in lst:
        str=""
        str = re.sub(r'[^a-zA-Z\s]', '', element)
        lst1.append(str)
    return lst1

#Normalization words
def stopwprds_removal(lst):
    lst1=list()
    for string in lst:
        text_tokens = word_tokenize(string)
        tokens_without_sw = [word for word in text_tokens if word.casefold() not in STOPWORDS]
        
        # Lemmatize the remaining words (reduce to their base form)
        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens_without_sw]

        # Join the lemmatized words back into a string
        normalized_text = " ".join(lemmatized_words)

        lst1.append(normalized_text)#str_t)
 
    return lst1

In [71]:
query = qry_set[idx] #get query
rel_docs = rel_set[idx] #relevant documents

corpus = list(doc_set.values())

# preprocess the documents
##removing special characters
corpus_without_sc = special_chars_removal(corpus)
## normalize texts
corpus_normalized = stopwprds_removal(corpus_without_sc)

tokenized_corpus = [doc.split(" ") for doc in corpus_normalized]
bm25 = BM25Okapi(tokenized_corpus)


In [72]:
%%markdown
# Top Five associated data

# Top Five associated data


In [73]:
# Process query
tokenized_query = query.split(" ")
print('Query ==> ', query, '\nRelevant documents IDs: ==> ', rel_docs)

doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores, len(doc_scores), len(doc_scores))

docs = bm25.get_top_n(tokenized_query, corpus_normalized, n=5)
docs

Query ==>  Methods of coding used in computerized index systems. 
Relevant documents IDs: ==>  [33, 43, 52, 56, 67, 79, 116, 122, 158, 319, 376, 433, 488, 492, 521, 529, 599, 667, 668, 669, 670, 672, 676, 677, 678, 680, 686, 688, 693, 696, 698, 699, 703, 705, 707, 708, 829, 1143]
[0. 0. 0. ... 0. 0. 0.] 1460 1460


['Truncated Search Key Title Index Long Philip L Kilgour Frederick G experiment showing search key derived title sufficiently specific efficient computerized interactive index file MARC II record',
 'Notation Coding Organic Compounds Geivandov E notation coding organic structure developed provides simple rational rule coding common cyclic fragment conventional unit coding regular structure benzol ring skeleton regular condensed coded sequence odd integer set rule coding regular system component universal notation organic compound code offered author designed cover important broad class compound conjugate bond framework specialized computerbased information retrieval capacity input internal machine language',
 'Method Construction MinimumRedundancy Codes Huffman David optimum method coding ensemble message consisting finite number member developed minimumredundancy code constructed way average number coding digit message minimized',
 'Thesaurofacet Multipurpose Retrieval Language Tool A